In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch
from nanoGPT.model import GPTConfig
from lightning_training import *

In [3]:
elite_database = pd.read_csv("./data/lichess_elite_database.csv", nrows=100000, delimiter=";", usecols=["result", "white_elo", "black_elo", "piece_uci"])
# elite_database = elite_database.sample(n = 10**7, random_state=42)
elite_database = elite_database.dropna()
elite_database

,white_elo,black_elo,result,piece_uci
0,2271,2428,1-0,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
1,2271,2428,1-0,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
2,2423,2438,1-0,Pd2d4 Ng8f6 Pc2c4 Pc7c6 Nb1c3 Pd7d5 Pe2e3 Pe7e...
3,2439,2276,1-0,Pe2e4 Pd7d5 Pe4d5 Ng8f6 Pc2c4 Pc7c6 Pd2d4 Pc6d...
4,2347,2430,1-0,Pd2d4 Pd7d5 Pc2c4 Pc7c6 Pc4d5 Pc6d5 Nb1c3 Nb8c...
...,...,...,...,...
99995,2285,2433,1-0,Pd2d4 Pg7g6 Pe2e4 Bf8g7 Nb1c3 Pd7d6 Bc1e3 Pa7a...
99996,2503,2508,1-0,Pe2e4 Pc7c5 Nb1c3 Pd7d6 Pf2f4 Pg7g6 Ng1f3 Bf8g...
99997,2310,2452,0-1,Pe2e4 Pc7c6 Pd2d4 Pd7d5 Pe4d5 Pc6d5 Bf1d3 Nb8c...
99998,2450,2312,1-0,Pd2d4 Pf7f5 Pg2g3 Ng8f6 Bf1g2 Pd7d6 Ng1f3 Pg7g...


In [4]:
# data_module = WeightedGamesDataModule(elite_database, test_size=0.01, num_workers=16)
# games = list(elite_database.piece_uci)
data_module = WeightedGamesDataModule(elite_database, batch_size=64, test_size=0.01, num_workers=12)

In [5]:
model_config = GPTConfig(
    block_size=301,
    vocab_size=len(data_module.tokenizer.vocab),
    n_layer=4,
    n_head=4,
    n_embd=256,
    bias=False,
)

pl_model = LightningGPTWeighted(model_config)
# pl_model = LightningGPT(model_config)

number of parameters: 4.19M


In [9]:
tensorboard_logger = pl.loggers.TensorBoardLogger(save_dir="./lightning_logs/strong_play/", name=f"W1-D0.5-L0")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=4,
    callbacks=[pl.callbacks.RichProgressBar()],
    logger=tensorboard_logger,
    precision=32,
    # default_root_dir=
    # fast_dev_run=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(
    model=pl_model,
    datamodule=data_module,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model         │ GPT                │  4.3 M │
│ 1 │ test_accuracy │ MulticlassAccuracy │      0 │
└───┴───────────────┴────────────────────┴────────┘

Trainable params: 4.3 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.3 M                                                                                                
Total estimated model params size (MB): 17

Output()